In [15]:
import numpy as np
import pandas as pd
import os
import glob
import mvpa_base_functions as bf
import pickle
from nilearn.input_data import NiftiLabelsMasker, NiftiMapsMasker, NiftiMasker
from nilearn.image import concat_imgs, load_img
import nibabel as nib

In [16]:
mask = 'fearnet'
phase = 'cond'
trial_block = 1
beta_folder = './sample_data/beta_files'# folder contains individual beta maps
savefile = f'./sample_data/discovery_{phase}_{mask}_{trial_block}block.pkl'
    
brain_file = './masks/whole_brain_mask_Sch7net400_subcortex_cerebellum_MNI152NLin2009cAsym_res-2space.nii.gz'
bimg = load_img(brain_file)
bdata = bimg.get_fdata().copy()
bdata[bdata>0] = 1
brain_img = nib.Nifti1Image(bdata, bimg.affine)

masker = NiftiMasker(brain_img, standardize=False)
val = masker.fit_transform(brain_file)

if mask=='fearnet':
    fearnet_file = './masks/fearNet_10regions_MNI152NLin2009cAsym_res-2_space.nii.gz'
    fear_img = load_img(fearnet_file)
    voxel_reg_ids = masker.transform(fear_img).reshape(-1)
    feature_index = voxel_reg_ids>0
elif mask=='wholebrain':
    fearnet_file = './masks/fearNet_10regions_dilute6mm_MNI152NLin2009cAsym_res-2_space.nii.gz'
    fear_img = load_img(fearnet_file)
    voxel_reg_ids = masker.transform(fear_img).reshape(-1)
    feature_index = voxel_reg_ids == 0
elif mask=='extendnet':
    extendnet_file = './masks/extendNet_24regions_MNI152NLin2009cAsym_res-2_space.nii.gz'
    extend_img = load_img(extendnet_file)
    voxel_reg_ids = masker.transform(extend_img).reshape(-1)
    
    feature_index = voxel_reg_ids>0
    
    

In [17]:
subj_dirs = glob.glob(os.path.join(beta_folder, phase, 'sub-*/'))
subj_ids = [i.split('/')[-2] for i in subj_dirs]
subj_ids.sort()

In [19]:
csp_data = []
csm_data = []
for csubj, subj in enumerate(subj_ids):
    bfile = os.path.join(beta_folder, phase, subj, f'con_CSp0{trial_block}.nii.gz')
    csp_dt = masker.transform(bfile).reshape(-1)
    csp_data.append(csp_dt[feature_index])
    
    bfile = os.path.join(beta_folder, phase, subj, f'con_CSm0{trial_block}.nii.gz')
    csm_dt = masker.transform(bfile).reshape(-1)
    csm_data.append(csm_dt[feature_index])
csp_data = np.stack(csp_data)
csm_data = np.stack(csm_data)

pickle.dump({'csp_data':csp_data,
             'csm_data':csm_data},
             open(savefile, 'wb'))